# PART 3: Connect with `City` Agent for remote SMC

In [1]:
%%javascript
document.title="Manufacturer1 Agent"

<IPython.core.display.Javascript object>

### Imports

In [2]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
from pprintpp import pprint

### Initialise the Agent Controller

In [3]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://manufacturer1-agent:3021 and an api key of adminApiKey


In [4]:
# Verify if VC "M1-isManufacturer-VC" already exists in wallet
vc = "M1-isManufacturer-VC"
credentials = await agent_controller.credentials.get_all()
if any(result["referent"] == vc for result in credentials["results"]):
    print("Credential {vc} already in wallet. Please proceed.".format(vc=vc))
else:
    print("Execute notebooks 01 to issue a VC to M1!")

Credential M1-isManufacturer-VC already in wallet. Please proceed.


### Start a Webhook Server

In [5]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [6]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    rfc_state = payload["rfc23_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : {routing} ({rfc})".format(routing=routing_state, rfc=rfc_state))
    if 'their_label' in payload: 
        print(f"Connection with : ", payload['their_label'])
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))


def prover_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    
    
    if state == "request_received":
        presentation_request = payload["presentation_request"]
        print("Recieved Presentation Request\n")
        print("\nRequested Attributes - Note the restrictions. These limit the credentials we could respond with\n")
        print(presentation_request["requested_attributes"])
    elif state == "presentation_sent":
        print("Presentation sent\n")
        
    elif state == "presentation_acked":
        print("Presentation has been acknowledged by the Issuer")
        
        
connection_listener = {"handler": connections_handler, "topic": "connections"}
listeners.append(connection_listener)

prover_listener = {"topic": "present_proof", "handler": prover_proof_handler}
listeners.append(prover_listener)

agent_controller.register_listeners(listeners)

## Accept Invitation

Copy an invitation object from another agent playing the role inviter (see the inviter_template recipe)

In [7]:
invitation = {
    '@id': '4b978b6d-8381-406e-bbe8-c7182ebe0dc8',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['842s8grD4E1x59xDyS3dnF8YwMqRFMXML5zW2KvakbZb'],
    'serviceEndpoint': 'https://80a67f24c0cb.ngrok.io',
}

In [8]:
auto_accept="true"
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  3a0a85a8-3d25-4242-91f3-79bba07f79f0
State :  invitation
Routing State : none (invitation-received)
Connection with :  City
Their Role :  inviter
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  3a0a85a8-3d25-4242-91f3-79bba07f79f0
State :  request
Routing State : none (request-sent)
Connection with :  City
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  3a0a85a8-3d25-4242-91f3-79bba07f79f0
State :  response
Routing State : none (response-received)
Connection with :  City
Their Role :  inviter
----------------------------------------------------------
response
-----------------------------------------

## Optional: Send Proposal

Propose a presentation to a verifier

In [ ]:
# TODO: Example proposal object below

# proposal_object = {
#   "auto_present": true,
#   "comment": "string",
#   "connection_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
#   "presentation_proposal": {
#     "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/presentation-preview",
#     "attributes": [
#       {
#         "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#         "mime-type": "image/jpeg",
#         "name": "favourite_drink",
#         "referent": "0",
#         "value": "martini"
#       }
#     ],
#     "predicates": [
#       {
#         "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#         "name": "high_score",
#         "predicate": ">=",
#         "threshold": 0
#       }
#     ]
#   },
#   "trace": false
# }

# proposal_response = await agent_controller.proofs.send_proposal(proposal_object)

## Fetch Presentation Records

Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. This could also be done through the present_proof listeners which have access to a presentation record in the payload.

In [9]:
# Step 2

# Optional Query parameters
verifier_connection_id = connection_id
thread_id = None
state = "request_received"
role = "prover"

proof_records_response = await agent_controller.proofs.get_records(verifier_connection_id, thread_id, state, role)
pprint(proof_records_response)

{
    'results': [
        {
            'connection_id': '3a0a85a8-3d25-4242-91f3-79bba07f79f0',
            'created_at': '2021-08-09 14:48:06.324610Z',
            'initiator': 'external',
            'presentation_exchange_id': 'a9bf341e-da74-4d21-a734-d986ff484359',
            'presentation_request': {
                'name': 'isManufacturer Proof Request',
                'non_revoked': {'to': 1628520443},
                'nonce': '544370537991510495111363',
                'requested_attributes': {
                    '0_isManufacturer_uuid': {
                        'name': 'isManufacturer',
                        'restrictions': [
                            {
                                'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1',
                            },
                        ],
                    },
                },
                'requested_predicates': {},
                'version': '1.0',
            },
            'presentation_

In [10]:
# Fetch records from response that were sent via connection_id
presentation_records = [r for r in proof_records_response["results"] if (r["connection_id"] == connection_id) is True] 

# Process only the first record:
presentation_record = presentation_records[0]
presentation_exchange_id = presentation_record["presentation_exchange_id"]

## Search For Available Credentials to Construct Presentation From

The presentation record can be used to query your agents wallet and return all credentials that could be used to construct valid presentation

In [11]:
def get_proof_request_requirements(presentation_record):
    """
    Returns dictionary with {<required-attribute>: <restrictions-of-attribute>} from presentation record
    """
    # Setup
    restrictions = {}
    presentation_request = presentation_record["presentation_request"]
    
    # Get required attributes and requirements for the individual attributes
    for attr_key, attr_val in presentation_request["requested_attributes"].items():
        restrictions[attr_val["name"]] = {}
        restrictions[attr_val["name"]]["requirements"] = attr_val["restrictions"][0] 
        restrictions[attr_val["name"]]["request_attr_name"] = attr_key
        
    return restrictions


def get_suitable_credentials(credentials_all, requirements):
    """
    Finds credentials in credentials_all that satisfy the requirements provided by the relying party.
    Returns dictionary with: {<attribute-name>: <suitable-credential>}, where the suitable-credential satisfies all requirements 
    """
    # Setup
    relevant_credentials = {}
    revealed = {}
    credentials = credentials_all["results"]
    
    # Iterate through attribute name and attribute requirements of relying party
    for name, conditions in requirements.items():
        
        req = conditions["requirements"]
        req_name = conditions["request_attr_name"]
        
        # Break if the required attribute name is not in any credential, or if all requirements (e.g., schema_id) are not within one credential
        if (any(name in cred["attrs"] for cred in credentials) is False) or (any(r in cred.keys() for r in req for cred in credentials) is False):
            continue

        # Iterate through credentials
        for cred in credentials:

            # Verify if requirement value (r_val) and credential value (cred[r_key]) match for required attribute (r_key)
            for r_key, r_val in req.items():
                try:
                    # Append cred to relevant_credentials if all requirements match
                    if (cred[r_key] == r_val) is True:
                        relevant_credentials[name] = cred
                        print(f"Attribute request for '{name}' can be satisfied by Credential with Referent -- {cred['referent']}")
                        revealed[req_name] = {"cred_id": cred["referent"], "revealed": True}
                except:
                    pass

    return relevant_credentials, revealed

In [12]:
# Get requirements of proof request
proof_attribute_requirements = get_proof_request_requirements(presentation_record)

# Verify all credentials of M1 and verify if they satisfy the requirements defined in proof_attribute_requirements
credentials = await agent_controller.credentials.get_all()
suitable_credentials, revealed = get_suitable_credentials(credentials, proof_attribute_requirements)
#pprint(credentials)
pprint(revealed)

Attribute request for 'isManufacturer' can be satisfied by Credential with Referent -- M1-isManufacturer-VC
{
    '0_isManufacturer_uuid': {
        'cred_id': 'M1-isManufacturer-VC',
        'revealed': True,
    },
}


In [40]:
# select credentials to provide for the proof
#credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
#print("Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n")

#attribute_by_reft = {}
#revealed = {}
#self_attested = {}
#predicates = {}

In [13]:
predicates = {}
self_attested = {}


In [14]:
# Note we are working on a friendlier api to abstract this away

print("\nGenerate the proof")
presentation = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(presentation)


Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_isManufacturer_uuid': {'cred_id': 'M1-isManufacturer-VC', 'revealed': True}}, 'self_attested_attributes': {}}


## Send Presentation

A presentation is sent in represent to a presentation record that has previously been created.

In [15]:
presentation_response = await agent_controller.proofs.send_presentation(presentation_exchange_id, presentation)


---------------------------------------------------------------------

Handle present-proof
Connection ID :  3a0a85a8-3d25-4242-91f3-79bba07f79f0
Presentation Exchange ID :  a9bf341e-da74-4d21-a734-d986ff484359
Protocol State :  presentation_sent
Agent Role :  prover
Initiator :  external

---------------------------------------------------------------------

Presentation sent


---------------------------------------------------------------------

Handle present-proof
Connection ID :  3a0a85a8-3d25-4242-91f3-79bba07f79f0
Presentation Exchange ID :  a9bf341e-da74-4d21-a734-d986ff484359
Protocol State :  presentation_acked
Agent Role :  prover
Initiator :  external

---------------------------------------------------------------------

Presentation has been acknowledged by the Issuer


## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [1]:
# Imports
import torch
import syft as sy

from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

In [2]:
# Step 2: Join duet of DS
duet_city = sy.duet("c119db5cdf423888d5a631b1f568b720")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: ff29b1c96f3d59db8d163ae3a61299ad

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [ ]:
# Copy and paste client_id and ping the city!

In [3]:
# Step 3: Create duet server
duet_m1 = sy.launch_duet()

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: bf2a7a2d63c30294c9bfa5c61ea9be9e

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("bf2a7a2d63c30294c9bfa5c61ea9be9e")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!


♫♫♫ > Duet Partner's Client ID:  5f521eaa1db551edb939fda2fba098f2



♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



## 2 - Secure Multi-Party Computation

In [4]:
# Step 8: test there is no data in the server atm
duet_m1.store.pandas

""


In [5]:
# Step 9: Publish secret data

x = torch.tensor([50,60,77]) # define torch
x.send(duet_m1, pointable=True, tags=["TEST_#1_from_m1_to_ds"], description="Dummy data") # send torch to duet_m1

x = torch.tensor([[1,2], [3,4]])
x.send(duet_m1, pointable=True, tags=["TEST_#2_from_m1_to_ds"], description="Test to multiply matrices")

In [6]:
# Step 10: verify that data was uploaded to the store
duet_m1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 469ba9871941459a8ae3de1b03713a0b>,[TEST_#1_from_m1_to_ds],Dummy data,<class 'torch.Tensor'>
1,<UID: f3f54225fc324bc18f4b59e7cce481d6>,[TEST_#2_from_m1_to_ds],Test to multiply matrices,<class 'torch.Tensor'>


In [7]:
# Step 11: Authorize the requests ds can make
duet_m1.requests.add_handler(action="accept")

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()